In [ ]:
from PIL import Image, ImageDraw
from collections import deque
import numpy as np
import os

# === Load & Process Maze ===

def load_maze_image(path):
    image = Image.open(path).convert("RGB")
    return image

def image_to_grid(image, invert=False):
    pixels = np.array(image)
    white = (pixels[:, :, 0] > 200) & (pixels[:, :, 1] > 200) & (pixels[:, :, 2] > 200)
    grid = white.astype(int)
    return 1 - grid if invert else grid


def find_start_end(grid):
    h, w = grid.shape
    for x in range(w):
        if grid[0, x] == 1:
            start = (0, x)
            break
    for x in range(w-1, -1, -1):
        if grid[h-1, x] == 1:
            end = (h-1, x)
            break
    return start, end

# === BFS ===

def bfs(grid, start, end):
    queue = deque([start])
    visited = set([start])
    parent = {}
    dirs = [(-1,0),(1,0),(0,-1),(0,1)]

    while queue:
        curr = queue.popleft()
        if curr == end:
            break
        for dx, dy in dirs:
            nx, ny = curr[0] + dx, curr[1] + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1]:
                if grid[nx][ny] == 1 and (nx, ny) not in visited:
                    queue.append((nx, ny))
                    visited.add((nx, ny))
                    parent[(nx, ny)] = curr
    return parent

def reconstruct_path(parent, start, end):
    path = []
    if end not in parent:
        return path
    current = end
    while current != start:
        path.append(current)
        current = parent[current]
    path.append(start)
    path.reverse()
    return path

# === Draw Solution ===

def draw_path(image, path, color=(255, 0, 0)):
    draw = ImageDraw.Draw(image)
    for y, x in path:
        draw.point((x, y), fill=color)
    return image

def solve_and_save(image_path, save_path):
    image = load_maze_image(image_path)

    # Try both normal and inverted path colors
    for invert in [False, True]:
        grid = image_to_grid(image, invert=invert)
        try:
            start, end = find_start_end(grid)
            parent = bfs(grid, start, end)
            path = reconstruct_path(parent, start, end)
            if path:
                solved_image = draw_path(image, path)
                solved_image.save(save_path)
                return
        except UnboundLocalError:
            continue  # Try next inversion
    print(f"No start/end found in {image_path}")


# === Batch Solve ===

input_folder = "data"
output_folder = "data-solved"
os.makedirs(output_folder, exist_ok=True)

for i in range(1, 50):
    fname = f"{i}.png"
    in_path = os.path.join(input_folder, fname)
    out_path = os.path.join(output_folder, fname)
    if os.path.exists(in_path):
        solve_and_save(in_path, out_path)




UnboundLocalError: cannot access local variable 'start' where it is not associated with a value